# Notebook Imports

In [20]:
from os import walk
from os.path import join
import pandas as pd

# Constants

In [35]:
EXAMPLE_FILE = '/Users/brendan/Desktop/Python/Machine Learning - Spam Email/SpamData/01_Processing/practice_email.txt'
ROOT_FILE = '/Users/brendan/Desktop/Python/Machine Learning - Spam Email'

SPAM_1_PATH = ROOT_FILE+'/SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_2_PATH = ROOT_FILE+'/SpamData/01_Processing/spam_assassin_corpus/spam_2'
EASY_NONSPAM_1_PATH = ROOT_FILE+'/SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
EASY_NONSPAM_2_PATH = ROOT_FILE+'/SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'

SPAM_CAT = 1
HAM_CAT = 0

# Reading Files

In [13]:
stream = open(EXAMPLE_FILE, encoding='latin-1')
message = stream.read()
stream.close()

print(type(message))
print(message)

<class 'str'>
From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassassin.taint.org
Received: fro

In [11]:
import sys
sys.getfilesystemencoding()

'utf-8'

In [14]:
stream = open(EXAMPLE_FILE, encoding='latin-1')

is_body = False
lines = []

for line in stream:
    if is_body:
        lines.append(line)
    elif line == '\n':
        is_body = True

stream.close()

email_body = '\n'.join(lines)
print(email_body)




Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

# Generator Functions

In [15]:
def generate_squares(N):
    for my_number in range(N):
        yield my_number ** 2

In [16]:
for i in generate_squares(3):
    print(i, end=' ->')

0 ->1 ->4 ->

In [17]:
for i in generate_squares(5):
    print(i, end=' ->')

0 ->1 ->4 ->9 ->16 ->

## Email body extraction

In [19]:
def email_body_generator(path):
    
    for root, dirnames, filenames in walk(path):
        for file_name in filenames:
            
            filepath = join(root, file_name)
            
            stream = open(filepath, encoding='latin-1')

            is_body = False
            lines = []

            for line in stream:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body = True

            stream.close()

            email_body = '\n'.join(lines)
            
            yield file_name, email_body

In [21]:
def df_from_directory(path, classification):
    rows = []
    row_names = []
    
    for file_name, email_body in email_body_generator(path):
        rows.append({'MESSAGE': email_body, 'CATEGORY': classification})
        row_names.append(file_name)
    
    return pd.DataFrame(rows, index=row_names)

In [33]:
spam_emails = df_from_directory(SPAM_1_PATH, 1)
spam_emails = spam_emails.append(df_from_directory(SPAM_2_PATH,1))
spam_emails.head()

,MESSAGE,CATEGORY
00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",1
00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,1
00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,1
00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,1
00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,1


In [34]:
spam_emails.shape

(1898, 2)

In [39]:
ham_emails = df_from_directory(EASY_NONSPAM_1_PATH,HAM_CAT)
ham_emails = ham_emails.append(df_from_directory(EASY_NONSPAM_2_PATH,HAM_CAT))
ham_emails.head(), ham_emails.shape

(                                                                                  MESSAGE  \
 00769.25bf9a767b5db0ed93f03c1637281663  \n\nIn a message dated 9/24/2002 11:24:58 AM, ...   
 01074.8590d61ac0aeeadb58dc2f2ba776c406  Hiya, I always seem to get errors when I do an...   
 02479.14365bcad3a60fcf24c5c1813f6291fb  \n\nI don't know how one can expect better and...   
 01417.ce7b07a2114218dbac682b599785820d  Tim Peters wrote:\n\n> I've run no experiments...   
 00357.d559b71616f64ba1d1c1e61a03644fd4  At 10:34 AM -0700 on 8/28/02, Jim Whitehead wr...   
 
                                         CATEGORY  
 00769.25bf9a767b5db0ed93f03c1637281663         0  
 01074.8590d61ac0aeeadb58dc2f2ba776c406         0  
 02479.14365bcad3a60fcf24c5c1813f6291fb         0  
 01417.ce7b07a2114218dbac682b599785820d         0  
 00357.d559b71616f64ba1d1c1e61a03644fd4         0  ,
 (3901, 2))

In [40]:
data = pd.concat([spam_emails, ham_emails])
print('SHape of entire dataframe is ', data.shape)
data.head()

SHape of entire dataframe is  (5799, 2)


,MESSAGE,CATEGORY
00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",1
00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,1
00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,1
00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,1
00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,1


In [41]:
data.tail()

,MESSAGE,CATEGORY
00609.dd49926ce94a1ea328cce9b62825bc97,"I'm one of the 30,000 but it's not working ver...",0
00957.e0b56b117f3ec5f85e432a9d2a47801f,Damien Morton quoted:\n\n>W3C approves HTML 4 ...,0
01127.841233b48eceb74a825417d8d918abf8,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\...",0
01178.5c977dff972cd6eef64d4173b90307f0,"Once upon a time, Manfred wrote :\n\n\n\n> I w...",0
00747.352d424267d36975a7b40b85ffd0885e,"If you run Pick, and then use the ""New FTOC"" b...",0


# Data Cleaning: Checking for Missing Values

In [46]:
# check if any message bodies are null
data['MESSAGE'].isnull().values.any()

False

In [51]:
len("")

0

In [47]:
type("")

str

In [49]:
my_var = None

In [50]:
type(my_var)

NoneType

In [54]:
# check if there are empty email (string length zero)
(data['MESSAGE'].str.len() == 0).any()

True

In [55]:
(data['MESSAGE'].str.len() == 0).sum()

3

In [59]:
# challenge: how to check the # of entries
data['MESSAGE'].isnull().sum()

0

## Locate empty emails

In [62]:
type(data['MESSAGE'].str.len() == 0)

pandas.core.series.Series

In [63]:
data[data['MESSAGE'].str.len() ==0].index



Index(['cmds', 'cmds', 'cmds'], dtype='object')

In [66]:
data.index.get_loc('cmds')

array([False, False, False, ..., False, False, False])

In [67]:
data[4608:4611]

,MESSAGE,CATEGORY
01121.d3a6706a1a5c4c927c4cc7a038367f62,"On August 7th, an entity known as ""iDEFENSE"" s...",0
01293.1bfec3fa6ca7c5fbc5bd7ffdfe2c2780,O'Reilly Open Source Convention\n\n-From the F...,0
00978.6ee6ee70e9126bba327faa762d37b3f9,Has anyone had much positive experience with P...,0
